# A Pythonic Object
## Object Representations

Every object-oriented language has at least one standard way of getting a string repre‐
sentation from any object. Python has two:
-  repr(): Return a string representing the object as the developer wants to see it.
-  str(): Return a string representing the object as the user wants to see it.

There are two additional special methods to support alternative representations of objects: __bytes__ and __format__. 

## Vector Class Redux

In [3]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y)) 
        
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self) 

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

myvec = Vector2d(7, 10)

In [6]:
for i in myvec: # using the __iter__
    print(i)

7.0
10.0


In [8]:
myvec.__repr__() # all the information we need to create this object

'Vector2d(7.0, 10.0)'

In [10]:
eval(myvec.__repr__()) == myvec

True

In [9]:
str(myvec) # a string representation of the object

'(7.0, 10.0)'

In [12]:
myvec.__bytes__()

b'd\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00$@'

In [14]:
Vector2d(1, 2) == Vector2d(1, 2) # using the __eq__

True

In [17]:
abs(myvec), math.hypot(7, 10) # using __abs__

(12.206555615733702, 12.206555615733702)

In [18]:
bool(myvec)

True

## classmethod Versus staticmethod
The staticmethod decorator changes a method so that it receives no special first argument. classmethod passes the class implicitly as the first argument.


In [21]:
class Demo:

    @classmethod
    def klassmeth(*args):
        return args

    @staticmethod
    def statmeth(*args):
        return args


In [22]:
Demo.klassmeth() # passes the class instance as 

(__main__.Demo,)

In [23]:
Demo.statmeth() # Demo.statmeth behaves just like a plain old function

()

The classmethod decorator is clearly useful, but I’ve never seen a compelling use case for staticmethod. If you want to define a
function that does not interact with the class, just define it in the module.

## Formatted Displays
The format() built-in function and the str.format() method delegate the actual formatting to each type by calling their .__format__(format_spec) method. 

In [25]:
brl = 1 / 2.43

In [26]:
brl

0.4115226337448559

In [27]:
format(brl, "0.4f")

'0.4115'

In [28]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

In [29]:
 format(42, 'b')

'101010'

In [30]:
format(2/3, '.1%')

'66.7%'

## A Hashable Vector2d
As defined, so far our Vector2d instances are unhashable, so we can’t put them in a set:

In [31]:
v1 = Vector2d(3, 4)

In [33]:
import traceback

try:
    hash(v1)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-33-b0a524b4faf1>", line 4, in <module>
    hash(v1)
TypeError: unhashable type: 'Vector2d'


To make a Vector2d hashable, we must implement __hash__ (__eq__ is also required, and we already have it). We also need to make vector instances immutable, as we’ve seen in 'What Is Hashable?' on page 65